## Playground for utilities to load, parse, and transform bank files


In [126]:
import pandas as pd
# import numpy as np
import json

from decimal import Decimal # To represent monetary values  https://docs.python.org/3/library/decimal.html  

In [127]:
# Data types for GL data
type transactionID = str   # Unique identifier for each transaction - TBD / maybe a hash of the transaction details?
type transactionDate = str # ... to be determined how to handle this
type transactionAmount = Decimal # 
type debitCreditIndicator = str # 'D' or 'C'
type transactionDescription = str
type accountID = str
type currencyUnit = str
type businessPartner = str
type Origin = str  # The original bank or credit card account that the transaction came from

# Can possibly be simplified by using @dataclass decorator
# Data type of GL item
class GLItem:
    def __init__(self, transactionID, transactionDate, transactionAmount, debitCreditIndicator, transactionDescription, accountID, currencyUnit, businessPartner, Origin):
        self.transactionID = transactionID
        self.transactionDate = transactionDate
        self.transactionAmount = transactionAmount
        self.debitCreditIndicator = debitCreditIndicator
        self.transactionDescription = transactionDescription
        self.accountID = accountID
        self.currencyUnit = currencyUnit
        self.businessPartner = businessPartner
        self.Origin = Origin

    def __str__(self):
        return f'{self.transactionID}, {self.transactionDate}, {self.transactionAmount}, {self.debitCreditIndicator}, {self.transactionDescription}, {self.accountID}, {self.currencyUnit}, {self.businessPartner}, {self.Origin}'

    def __repr__(self):
        return f'{self.transactionID}, {self.transactionDate}, {self.transactionAmount}, {self.debitCreditIndicator}, {self.transactionDescription}, {self.accountID}, {self.currencyUnit}, {self.businessPartner}, {self.Origin}'

In [ ]:
# Read Wells Fargo test file into data frame
wfc_ColumnNames = ['Date', 'Amount', 'Unknown', 'CheckNo', 'Description']
# wfc_ColumnDataTypes = {'Amount': np.float16, 'Unknown': str, 'CheckNo': np.int16, 'Description': str}
wfc_RawData = pd.read_csv('Bank-Files/WF-test.csv', index_col=None, parse_dates=True, 
                         names=wfc_ColumnNames)
wfc_RawData.head(5)

In [ ]:
# Read bank descriptor
with open('BankDescriptor-WFC.json', 'r') as file:
    wfc_bankDescriptor = json.load(file)

print(wfc_bankDescriptor)

In [ ]:
# Read account determination (JSON file) into dictionary

with open(wfc_bankDescriptor['AccountDeterminationFile'], 'r') as file:
    wfc_accountDetermination = json.load(file)

print(wfc_accountDetermination)

In [ ]:
print(wfc_accountDetermination['PITCAIRN HOMEOWN Payment'])

In [ ]:

for key, value in wfc_accountDetermination.items():
    print(f"String: {key}, Account: {value["Account"]}")

In [ ]:

for index, row in wfc_RawData.head(5).iterrows():
    print(f"Index: {index}")
    print(row.Date, row.Amount, row.Description)

In [134]:
# Another way to loop the data frame
# Assuming df is your DataFrame
#def process_row(row):
#    print(row)

#wfc_RawData.apply(process_row, axis=1)

In [ ]:
# Loop through the data frame and determine the account
for index, row in wfc_RawData.head(5).iterrows():
    print(f"Index: {index}")
    print(row.Date, row.Amount, row.Description)
    for key, value in wfc_accountDetermination.items():
        #print(f"String: {key}, Account: {value}")
        if key in row.Description:
            print(f"Account determination: String: {key}, Account: {value["Account"]}, Business Partner: {value["BP"]}")
            break
    # handle case when no account is found: ...
    

In [136]:
class AccountDetermination:
    def __init__(self, json_file):
        with open(json_file, 'r') as file:
            self.account_determination = json.load(file)

    def determine_account(self, description: str):
        for substring, account in self.account_determination.items():
            if substring in description:
                return account
        return None

# Example usage:
# account_det = AccountDetermination('AccountDetermination-WFC.json')
# account = account_det.determine_account('INTEREST PAYMENT')
# print(account)

In [ ]:
#
account_determination_wfc = AccountDetermination('AccountDetermination-WFC.json')

for index, row in wfc_RawData.head(15).iterrows():
    print(f"Index: {index}", row.Date, row.Amount, row.Description, 
          f"Account: {account_determination_wfc.determine_account(row.Description)}")
    accounts = account_determination_wfc.determine_account(row.Description)
    if accounts is None:
        print(f"Account not found for description: {row.Description}")
        continue
    accountID = accounts['Account']
    businessPartner = accounts['BP']
    print(f"Account: {accountID}, Business Partner: {businessPartner}")
    transactionDate = row.Date
    transactionAmount = row.Amount
    print(f"Transaction Date: {transactionDate}, Transaction Amount: {transactionAmount}")
    transactionDescription = row.Description
    print(f"Transaction Description: {transactionDescription}")
    # set the currency unit
    currencyUnit = 'USD'
    # Set the debit or credit indicator (this is based on GL, no how banks handle it - just the opposite)
    # For a regular bank account, a deposit is a debit and a withdrawal is a credit to the bank balance sheet account
    if transactionAmount >= 0:
        debitCreditIndicator = 'D' # Debit to bank balance sheet account
    else:
        debitCreditIndicator = 'C' # Credit to bank balance sheet account
    # Set the origin
    Origin = wfc_bankDescriptor['BankAccountDescription']

    # set the transactionID
    transactionID = 'TBD'

    # Create a GL item
    gl_item = GLItem(transactionID, transactionDate, transactionAmount, debitCreditIndicator, transactionDescription, accountID, currencyUnit, businessPartner, Origin)
    print("--------------------")
    print(gl_item)
    print("--------------------")

